# Sistemas Inteligentes 2021/2022

## Mini-projeto 1: Pacman comilão

<img src="pacman.png" alt="Drawing" style="width: 100px;"/>

## Grupo: 35

### Elementos do Grupo

Número: 55164        Nome: João Lago     
Número: 56947        Nome: Lara Marques    
Número: 56919        Nome: Maria Elena Munteanu     

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

## Representação dos estados

(descreva aqui, textualmente, como decidiu representar os estados em Python; ilustre nas células de código abaixo a representação em Python de um estado à sua escolha)

Representámos o estado do pacman por:
  - Um tuplo de coordenadas (x,y) com a sua posição
  - Um dicionário em que as chaves são os nomes das diferentes pastilhas e os valores são listas de tuplos com as coordenadas de cada pastilha, podendo haver mais do que uma pastilha de cada tipo
  - Um inteiro com os pontos do pacman
  - Um inteiro com o número de células percorridas
  - Um dicionário em que as chaves são os tuplos com as coordenadas das células percorridas pelo pacman e os valores são o número de vezes que o pacman passou por essa célula


Igualdade de estados:
  - Dados dois estados A e B consideramos A igual a B se a posição no estado A for igual à posição do estado B

Comparação de estados:
  - Dados dois estados A e B consideramos A maior que B se a quantidade de pontos do estado A for maior que a quantidade de pontos do estado B
  - Dados dois estados A e B consideramos A menor que B se a quantidade de pontos do estado A for menor que a quantidade de pontos do estado B

In [25]:
# se definiu uma classe para representar os estados, inclua aqui o código Python correspondente

class EstadoPacman :

    def __init__(self, posicao,pastilhas,pontos,t,caminhoPercorrido):
        """ Construtor da classe EstadoPacman"""
        self.pacman = (posicao,pastilhas,pontos,t,caminhoPercorrido) 

    def __eq__(self,other):
        "Verifica se os estados são iguais"
        return isinstance(other, EstadoPacman) and self.pacman[0]==other.pacman[0]

    def __lt__(self,other):
        "Verifica qual o estado menor"
        return self.pacman[2]<other.pacman[2]

    def __gt__(self,other):
        "Verifica qual o estado maior"
        return self.pacman[2]>other.pacman[2]

    def __hash__(self):
        "Retorna o hash value"
        hashableObj = (self.pacman[0])
        return hash(hashableObj)

    def __repr__(self):
        "Devolve a representação do estado"
        return "<Pacman {} --- Pastilhas {} ---- Pontos {} ------ T {} ----- Caminho {}>".format(self.pacman[0],self.pacman[1],self.pacman[2],self.pacman[3],self.pacman[4])

In [26]:
# representação de um estado à sua escolha  
p1 = EstadoPacman((1,1),{"N":[(3,7),(2,1)], "D":[(1,5)], "C":[(7,3)]},0,0,{(1,1):1})
print(str(p1))

<Pacman (1, 1) --- Pastilhas {'N': [(3, 7), (2, 1)], 'D': [(1, 5)], 'C': [(7, 3)]} ---- Pontos 0 ------ T 0 ----- Caminho {(1, 1): 1}>


## Formulação do problema

(explique textualmente como decidiu formular em Python o seu problema)

O Problema do Pacman Comilão está encapsulado na classe PacmanPastilhas que recebe a posição inicial do pacman, o total de pontos que o pacman precisa de obter para alcançar o objetivo do problema, o conjunto de pastilhas, o conjunto de obstáculos e a dimensão do mundo, sendo assim inicializada com o estado do Pacman que recebe toda a informação mutável do problema (a posição do pacman, o conjunto de pastilhas, os pontos do pacman, o número de células percorridas, as células que passou e quantas vezes passou por elas), e atributos imutáveis do problema (obstáculos, dimensão do quadro e o objetivo) tornam-se propriedades do problema, e tem também um atributo directions que é um dicionário com as possíveis direções do pacman e as mudanças a fazer na posição do pacman.

Dentro da classe PacmanPastilhas temos as seguintes funções:

  - calculaPontos que calcula os pontos que o pacman deve receber quando come uma pastilha dependendo da pastilha que encontrou
  - result que devolve um estado resultante de uma certa ação (de self.actions(state)) no estado dado 
  - goal_test que verifica se um certo estado satisfaz o objetivo
  - actions que calcula as ações que podem ser feitas no estado dado
  - path_cost que devolve o custo de um caminho entre state e state2
  - display que representa o estado
  - display_trace que representa o caminho feito desde o início


In [27]:
from searchPlus import *
from copy import deepcopy
import timeit

In [28]:
def exec(p,estado,accoes):
    custo = 0
    for a in accoes:
        seg = p.result(estado,a)
        custo = p.path_cost(custo,estado,a,seg)
        estado = seg
    p.display(estado)
    print('Custo:',custo)
    print('Goal?',p.goal_test(estado))
    return (estado,custo)


def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def quadro(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    return line(x,y,0,1,length) | line(x+length-1,y,0,1,length) | line(x,y,1,0,length) | line(x,y+length-1,1,0,length)

In [29]:
class PacmanPastilhas(Problem):
    """Encontrar um caminho numa grelha 2D com obstáculos. Os obstáculos são células (x, y)."""

    def __init__(self, initial=(1,1), goal=10,pastilhas={},obstacles={}, dim=10):
        """
            Construtor da classe PacmanPastilhas
        """
        estadoPacman = EstadoPacman(initial,pastilhas,0,0,{initial:1})
        super().__init__(estadoPacman,goal)
        self.obstacles=obstacles
        self.dim = dim

    directions = {"N":(0, -1), "W":(-1, 0), "E":(1,  0),"S":(0, +1)}

    def calculaPontos(self,dir,t):
        """
        Esta função tem como objetivo calcular os pontos de um dado estado
        """
        if dir == 'N': 
            return 1
        elif dir == 'D':
            return max(0,5 - t)
        else:
            return t
               
    def result(self, state, action): 
        """
            Esta função tem como objetivo gerar um novo estado resultante de aplicar a acção action ao estado state
        """
        (x,y) = state.pacman[0]
        (dx,dy) = self.directions[action]
        novaPosicao =(x+dx,y+dy)
        
        novoCaminhoPercorido = deepcopy(state.pacman[4])
        novasPastilhas = deepcopy(state.pacman[1])
        
        novosPontos = state.pacman[2]
        novoT = state.pacman[3]+1

        if novaPosicao in state.pacman[4]:
            novoCaminhoPercorido[novaPosicao] += 1 
        else:
            novoCaminhoPercorido[novaPosicao] = 1 
             
        listaPast = list(novasPastilhas.items())
        for(letraPast,coordPast) in listaPast:
            if novaPosicao in coordPast:
                novosPontos += self.calculaPontos(letraPast,novoT)
                novasPastilhas[letraPast].remove(novaPosicao)
        novoEstado = EstadoPacman(novaPosicao,novasPastilhas,novosPontos,novoT,novoCaminhoPercorido)  
        return novoEstado
        
    
    def goal_test(self, state):
        """
           Esta função tem como objetivo avaliar se o estado state cumpre o goal
        """
        if state.pacman[2] >= self.goal:
            return True 
        else:
            return False
    

    def actions(self, state):
        """
            Esta funcao tem como objectivo calcular todas as posições o pacman pode fazer(possiveis) a partir de uma posição presente no estado state
        """
        x, y = state.pacman[0]
        result = [act for act in self.directions.keys() if (x+self.directions[act][0],y+self.directions[act][1]) not in self.obstacles]
        return result
    
    def path_cost(self,c,state,action,state2):
        """
           Esta funcao tem como objetivo calcular o path cost para um determinado estado state 
        """
        if state2.pacman[0] in state.pacman[4]:
            return c+state.pacman[4][state2.pacman[0]]
        else: 
            return c+1
    
    def display(self, state):
        """
            Esta funcao printa o display do estado do problema
        """
        output=""  
        for j in range(self.dim):
            for i in range(self.dim):
                if state.pacman[0] ==(i,j):
                    ch = '@'
                elif (i,j) in self.obstacles:
                    ch = "="
                else:
                    ch = "."
                listaPast = list(state.pacman[1].items())
                for elem in range(len(listaPast)):
                    if (i,j) in listaPast[elem][1]:
                        ch = listaPast[elem][0]    
                output += ch + " "
            output += "\n"
        print(output)
        
        
    def display_trace(self,d,plan):
        """
            Esta funcao printa o display do caminho até ao estado final
        """
        path = set()
        st = self.initial
        pastilhas = {}
        for a in plan:
            st = self.result(st,a)
            path.add(st.pacman[0])
            pastilhas = st.pacman[1]
        output=""
        for j in range(self.dim):
            for i in range(self.dim):
                if self.initial.pacman[0] ==(i,j):
                    ch = '@'
                elif self.goal==(i,j):
                    ch = "*"
                elif (i,j) in self.obstacles:
                    ch = "="
                elif (i,j) in path:
                    ch = '+'
                else:
                    ch = "."
                listaPast = list(pastilhas.items())
                for elem in range(len(listaPast)):
                    if (i,j) in listaPast[elem][1]:
                        ch = listaPast[elem][0] 
                output += ch + " "
            output += "\n"
        print(output)
    

## Criação de estados e do problema

(Mostrem que o código está a funcionar, construindo instâncias da classe **PacmanPastilhas**, fazendo display dos estados, verificando o teste do estado final, gerando as ações para alguns estados, executando ações a partir de alguns estados e gerando novos estados e mostrando a evolução dos custos; verificando que os estados não se modificam com as ações (são gerados novos estados) e que a igualdade e a comparação entre estados funciona. Mostrem que a execução de sequências de ações está a funcionar bem.)

In [30]:
f1 = quadro(0,0,10)
l = line(2,2,1,0,6)
c = line(2,3,0,1,4)
obstaculos = f1|l|c
p1 = {"N":[(3,7),(2,1)], "D":[(4,5)], "C":[(7,3)]}
p2 = {'N':[(1,2)], 'D':[(5,6), (1,4)], 'C':[(7,4), (5,3)]}

As Instâncias recomendadas para o algoritmo de Profundidade Primeiro em Árvore são: prob2 e prob3, uma vez que o tempo de execução pode ser muito grande.

Criámos instâncias com objetivos diferentes e pastilhas diferentes para podermos observar melhor as diferenças entre os algoritmos

In [31]:
#-----------------------------Instâncias PacmanPastilhas-----------------------------#
prob1 = PacmanPastilhas((1,1),1,p1,obstaculos,10) 
prob2 = PacmanPastilhas((1,1),2,p2,obstaculos,10) 

prob3 = PacmanPastilhas((1,1),3,p2,obstaculos,10)
prob4 = PacmanPastilhas((1,1),10,p1,obstaculos,10)

In [32]:
def printaEstadoTeste(problem,state,custo=0):
    """
        Esta função printa o display, os pontos, os custos e o teste do estado final de um state
    """
    problem.display(state)
    print('Pontos ->',state.pacman[2])
    print('Custos ->', custo)
    print('É o estado final? ',problem.goal_test(state))

In [33]:
print('Exemplo 1')
print('Estado 1 -- Objetivo 1')
state1 = prob1.initial
custo = 0
printaEstadoTeste(prob1,state1,custo)
act1 = prob1.actions(state1)
print('Ações Possíveis ->',act1, end='\n \n')

print('Estado 2 -- Objetivo 3')
print('State 1 antes do Result gerar novo estado -- ',str(state1))
state2 = prob1.result(state1,act1[0])
print('State 1 (estado Velho) depois da função result -- ',str(state1))
print('State 2 (novo Estado) depois da funçao result -- ',str(state2))

custo += prob1.path_cost(custo,state1,act1[0],state2)
printaEstadoTeste(prob1,state2,custo)


print('Igualdade de estados')
print('O estado velho(state1) é igual ao novo(state2)? ->', state1 == state2)

print('Comparação de estados')
print('O estado velho(state1) é maior que o estado novo(state2)? ->', state1 > state2)
print('O estado velho(state1) é menor que o estado novo(state2)? ->', state1 < state2)

print('-------------------------------------------------------------------------------------------')


Exemplo 1
Estado 1 -- Objetivo 1
= = = = = = = = = = 
= @ N . . . . . . = 
= . = = = = = = . = 
= . = . . . . C . = 
= . = . . . . . . = 
= . = . D . . . . = 
= . = . . . . . . = 
= . . N . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Pontos -> 0
Custos -> 0
É o estado final?  False
Ações Possíveis -> ['E', 'S']
 
Estado 2 -- Objetivo 3
State 1 antes do Result gerar novo estado --  <Pacman (1, 1) --- Pastilhas {'N': [(3, 7), (2, 1)], 'D': [(4, 5)], 'C': [(7, 3)]} ---- Pontos 0 ------ T 0 ----- Caminho {(1, 1): 1}>
State 1 (estado Velho) depois da função result --  <Pacman (1, 1) --- Pastilhas {'N': [(3, 7), (2, 1)], 'D': [(4, 5)], 'C': [(7, 3)]} ---- Pontos 0 ------ T 0 ----- Caminho {(1, 1): 1}>
State 2 (novo Estado) depois da funçao result --  <Pacman (2, 1) --- Pastilhas {'N': [(3, 7)], 'D': [(4, 5)], 'C': [(7, 3)]} ---- Pontos 1 ------ T 1 ----- Caminho {(1, 1): 1, (2, 1): 1}>
= = = = = = = = = = 
= . @ . . . . . . = 
= . = = = = = = . = 
= . = . . . . C . = 
= . = . . . .

In [34]:
print('Exemplo 2')
print('Estado 1 -- Objetivo 1')
state1 = prob2.initial
custo = 0
printaEstadoTeste(prob2,state1,custo)
acoes = ['E','E','E','E','E','E','E','S','S','W']

print('Estado 2 -- Objetivo 3')
print('State 1 antes do exec gerar novo estado -- ',str(state1))
state2 = exec(prob2,state1,acoes)
print('State 1 (estado Velho) depois da função exec -- ',str(state1))
print('State 2 (novo Estado) depois da funçao exec -- ',str(state2[0]))

custo += state2[1]
printaEstadoTeste(prob2,state2[0],custo)


print('Igualdade de estados')
print('O estado velho(state1) é igual ao novo(state2)? ->', state1 == state2[0])

print('Comparação de estados')
print('O estado velho(state1) é maior que o estado novo(state2)? ->', state1 > state2[0])
print('O estado velho(state1) é menor que o estado novo(state2)? ->', state1 < state2[0])

Exemplo 2
Estado 1 -- Objetivo 1
= = = = = = = = = = 
= @ . . . . . . . = 
= N = = = = = = . = 
= . = . . C . . . = 
= D = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Pontos -> 0
Custos -> 0
É o estado final?  False
Estado 2 -- Objetivo 3
State 1 antes do exec gerar novo estado --  <Pacman (1, 1) --- Pastilhas {'N': [(1, 2)], 'D': [(5, 6), (1, 4)], 'C': [(7, 4), (5, 3)]} ---- Pontos 0 ------ T 0 ----- Caminho {(1, 1): 1}>
= = = = = = = = = = 
= . . . . . . . . = 
= N = = = = = = . = 
= . = . . C . @ . = 
= D = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Custo: 10
Goal? False
State 1 (estado Velho) depois da função exec --  <Pacman (1, 1) --- Pastilhas {'N': [(1, 2)], 'D': [(5, 6), (1, 4)], 'C': [(7, 4), (5, 3)]} ---- Pontos 0 ------ T 0 ----- Caminho {(1, 1): 1}>
State 2 (novo Estado) depois da funçao exec --  <Pacman (7, 3) --- Pasti

## Teste de procura de solução

(utilização de algoritmos de procura aprendidos nas aulas e comparação dos resultados ao nível de tempo de execução e solução obtida; comente aqui os resultados obtidos e o que observa)

In [35]:
#------------------------------Algoritmos------------------------------#

#-----------------------------Profundidade-----------------------------#

def depth_first_graph_search(problem): 
    """Search the deepest nodes in the search tree first."""
    return graph_search(problem, Stack())

def depth_first_tree_search(problem): 
    """Search the deepest nodes in the search tree first."""
    return tree_search(problem, Stack())

#--------------------------------Largura--------------------------------#

def breadth_first_graph_search(problem):
    """Search the deepest nodes in the search tree first."""
    return graph_search(problem, FIFOQueue())

def breadth_first_tree_search(problem): 
    """Search the shallowest nodes in the search tree first."""
    return tree_search(problem, FIFOQueue())

#----------------------------Custo Uniforme----------------------------#

def uniform_cost_search(problem): 
    """[Figure 3.14]"""
    return best_first_graph_search(problem, lambda node: node.path_cost)

#----------------------Aprofundamento Progressivo-----------------------#

def iterative_deepening_search(problem):
    """[Figure 3.18]"""
    for depth in range(sys.maxsize):
        result = depth_limited_search(problem, depth)
        if result != 'cutoff':
            return result 

In [36]:
def mostraProblema(problema,algoritmo,nome):
    """
        Esta função trata de mostrar todo o problema de acordo com o algoritmo dado
    """
    problema.display(problema.initial)
    start = timeit.default_timer()
    resultado = algoritmo(problema)
    if resultado:
        print("Solução ", nome," com custo", str(resultado.path_cost)+":")
        print('Objetivo --> ',problema.goal)
        print('Pontos --> ',resultado.state.pacman[2])
        print('Solução --> ',resultado.solution())
        stop = timeit.default_timer()
        print('Tempo de Execução --> ', stop - start, end='\n\n')
        problema.display_trace(10,resultado.solution())
    else:
        print('Sem Solução')


### Problema 1 
Algoritmo Profundidade Primeiro em Árvore

In [37]:
mostraProblema(prob2,depth_first_tree_search,'Prof-prim (Árvore)')

= = = = = = = = = = 
= @ . . . . . . . = 
= N = = = = = = . = 
= . = . . C . . . = 
= D = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Prof-prim (Árvore)  com custo 3:
Objetivo -->  2
Pontos -->  3
Solução -->  ['S', 'S', 'S']
Tempo de Execução -->  0.0005666270001256635

= = = = = = = = = = 
= @ . . . . . . . = 
= + = = = = = = . = 
= + = . . C . . . = 
= + = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 



##### Observações sobre o problema 1 

Utilizámos este problema para representar melhor o comportamento do Algoritmo de Profundidade Primeiro em Árvore, uma vez que não o vamos utilizar no problema com o objetivo mais elevado uma vez que este pode ter um tempo de execução maior ou mesmo criar um loop infinito.

Neste problema, podemos observar que o tempo de execução do algoritmo é bastante pequeno e como o algoritmo começa a procura mais à esquerda, apesar de poder passar por toda a árvore e entrar em ciclo infinito, neste caso não o faz pois encontra rapidamente a solução com o custo homogéneo. 

### Problema 2
Algoritmo Profundidade Primeiro em Árvore

In [38]:
mostraProblema(prob3,depth_first_tree_search,'Prof-prim (Árvore)')

= = = = = = = = = = 
= @ . . . . . . . = 
= N = = = = = = . = 
= . = . . C . . . = 
= D = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Prof-prim (Árvore)  com custo 3:
Objetivo -->  3
Pontos -->  3
Solução -->  ['S', 'S', 'S']
Tempo de Execução -->  0.0005732539998462016

= = = = = = = = = = 
= @ . . . . . . . = 
= + = = = = = = . = 
= + = . . C . . . = 
= + = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 



Algoritmo Profundidade Primeiro em Grafo

In [39]:
mostraProblema(prob3,depth_first_graph_search,'Prof-prim (Grafo)')

= = = = = = = = = = 
= @ . . . . . . . = 
= N = = = = = = . = 
= . = . . C . . . = 
= D = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Prof-prim (Grafo)  com custo 3:
Objetivo -->  3
Pontos -->  3
Solução -->  ['S', 'S', 'S']
Tempo de Execução -->  0.0007317330000660149

= = = = = = = = = = 
= @ . . . . . . . = 
= + = = = = = = . = 
= + = . . C . . . = 
= + = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 



Algoritmo Largura Primeiro em Árvore

In [40]:
mostraProblema(prob3,breadth_first_tree_search,'Largura-prim (Árvore)')

= = = = = = = = = = 
= @ . . . . . . . = 
= N = = = = = = . = 
= . = . . C . . . = 
= D = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Largura-prim (Árvore)  com custo 3:
Objetivo -->  3
Pontos -->  3
Solução -->  ['S', 'S', 'S']
Tempo de Execução -->  0.001556738000090263

= = = = = = = = = = 
= @ . . . . . . . = 
= + = = = = = = . = 
= + = . . C . . . = 
= + = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 



Algoritmo Largura Primeiro em Grafo

In [41]:
mostraProblema(prob3,breadth_first_graph_search,'Largura-prim (Grafo)')

= = = = = = = = = = 
= @ . . . . . . . = 
= N = = = = = = . = 
= . = . . C . . . = 
= D = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Largura-prim (Grafo)  com custo 3:
Objetivo -->  3
Pontos -->  3
Solução -->  ['S', 'S', 'S']
Tempo de Execução -->  0.0012851320000208943

= = = = = = = = = = 
= @ . . . . . . . = 
= + = = = = = = . = 
= + = . . C . . . = 
= + = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 



Algoritmo Custo Uniforme em Grafo

In [42]:
mostraProblema(prob3,uniform_cost_search,'Custo Uniforme (Grafo)')

= = = = = = = = = = 
= @ . . . . . . . = 
= N = = = = = = . = 
= . = . . C . . . = 
= D = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Custo Uniforme (Grafo)  com custo 3:
Objetivo -->  3
Pontos -->  3
Solução -->  ['S', 'S', 'S']
Tempo de Execução -->  0.0014830540001185

= = = = = = = = = = 
= @ . . . . . . . = 
= + = = = = = = . = 
= + = . . C . . . = 
= + = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 



Algoritmo Aprofundamento Progressivo

In [39]:
mostraProblema(prob3,iterative_deepening_search,'Aprofundamento Progressivo')

= = = = = = = = = = 
= @ . . . . . . . = 
= N = = = = = = . = 
= . = . . C . . . = 
= D = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Aprofundamento Progressivo  com custo 3:
Objetivo -->  3
Pontos -->  3
Solução -->  ['S', 'S', 'S']
Tempo de Execução -->  0.0011802559999978257

= = = = = = = = = = 
= @ . . . . . . . = 
= + = = = = = = . = 
= + = . . C . . . = 
= + = . . . . C . = 
= . = . . . . . . = 
= . = . . D . . . = 
= . . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 



##### Observações sobre o problema 2
Neste problema, podemos observar que apesar de todos os algoritmos adotarem métodos diferentes de processar o problema, chegam todos à mesma solução. O tempo de execução varia entre 0.0005s (Profundidade Primeiro em Árvore) e 0.0016s (Largura Primeiro em Árvore). Nenhum dos algoritmos entrou num ciclo infinito. Como todos os algoritmos correm com os custos homogéneos, a solução é optimal.

Como o algoritmo Profundidade Primeiro em Árvore expande primeiro os nós que foram visitados mais recentemente, expandindo o ramo mais à esquerda e a solução não está muito profunda, a procura é menos demorada, não sendo necessário processar a árvore toda.

O algoritmo Largura Primeiro em Árvore expande primeiro os nós que foram visitados primeiro, ou seja os nós com menor profundidade, como a solução não se encontra nas camadas mais à superfície, este demora mais tempo a chegar ao ramo da solução.

Nos algoritmos:
   - Profundidade Primeiro em Grafo (0.0007s)
   - Aprofundamento Progressivo (0.0012s)
   - Largura Primeiro em Grafo (0.0013s)
   - Custo Uniforme (0.0015s)

No Algoritmo Custo Uniforme o tempo é mais elevado uma vez apesar de um estado não poder ser expandido pela segunda vez, este pode ser revisitado, sendo que a procura depende do custo do caminho.

### Problema 3
Algoritmo Profundidade Primeiro em Árvore

In [47]:
try:
    mostraProblema(prob4,depth_first_tree_search,'Prof-prim (Árvore)')
except KeyboardInterrupt:
    print('Entrou num ciclo infinito! ')

= = = = = = = = = = 
= @ N . . . . . . = 
= . = = = = = = . = 
= . = . . . . C . = 
= . = . . . . . . = 
= . = . D . . . . = 
= . = . . . . . . = 
= . . N . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Entrou num ciclo infinito! 


Algoritmo Profundidade Primeiro em Árvore

In [48]:
mostraProblema(prob4,depth_first_graph_search,'Prof-prim (Grafo)')

= = = = = = = = = = 
= @ N . . . . . . = 
= . = = = = = = . = 
= . = . . . . C . = 
= . = . . . . . . = 
= . = . D . . . . = 
= . = . . . . . . = 
= . . N . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Prof-prim (Grafo)  com custo 28:
Objetivo -->  10
Pontos -->  28
Solução -->  ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'N', 'N', 'W', 'W', 'W', 'W', 'W', 'N', 'N', 'E', 'E', 'E', 'E', 'N']
Tempo de Execução -->  0.010506539000289195

= = = = = = = = = = 
= @ N . . . . . . = 
= + = = = = = = . = 
= + = . . . . + . = 
= + = + + + + + . = 
= + = + D . . . . = 
= + = + + + + + + = 
= + . N . . . . + = 
= + + + + + + + + = 
= = = = = = = = = = 



Algoritmo Largura Primeiro em Árvore

In [49]:
mostraProblema(prob4,breadth_first_tree_search,'Largura-prim (Árvore)')

= = = = = = = = = = 
= @ N . . . . . . = 
= . = = = = = = . = 
= . = . . . . C . = 
= . = . . . . . . = 
= . = . D . . . . = 
= . = . . . . . . = 
= . . N . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Largura-prim (Árvore)  com custo 10:
Objetivo -->  10
Pontos -->  11
Solução -->  ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W']
Tempo de Execução -->  0.1368181560001176

= = = = = = = = = = 
= @ + + + + + + + = 
= . = = = = = = + = 
= . = . . . . + + = 
= . = . . . . . . = 
= . = . D . . . . = 
= . = . . . . . . = 
= . . N . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 



Algoritmo Largura Primeiro em Grafo

In [50]:
mostraProblema(prob4,breadth_first_graph_search,'Largura-prim (Grafo)')

= = = = = = = = = = 
= @ N . . . . . . = 
= . = = = = = = . = 
= . = . . . . C . = 
= . = . . . . . . = 
= . = . D . . . . = 
= . = . . . . . . = 
= . . N . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Largura-prim (Grafo)  com custo 10:
Objetivo -->  10
Pontos -->  11
Solução -->  ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W']
Tempo de Execução -->  0.0034288709998691047

= = = = = = = = = = 
= @ + + + + + + + = 
= . = = = = = = + = 
= . = . . . . + + = 
= . = . . . . . . = 
= . = . D . . . . = 
= . = . . . . . . = 
= . . N . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 



Algoritmo Custo Uniforme em Grafo

In [51]:
mostraProblema(prob4,uniform_cost_search,'Custo Uniforme (Grafo)')

= = = = = = = = = = 
= @ N . . . . . . = 
= . = = = = = = . = 
= . = . . . . C . = 
= . = . . . . . . = 
= . = . D . . . . = 
= . = . . . . . . = 
= . . N . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Custo Uniforme (Grafo)  com custo 10:
Objetivo -->  10
Pontos -->  11
Solução -->  ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W']
Tempo de Execução -->  0.004891725999641494

= = = = = = = = = = 
= @ + + + + + + + = 
= . = = = = = = + = 
= . = . . . . + + = 
= . = . . . . . . = 
= . = . D . . . . = 
= . = . . . . . . = 
= . . N . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 



Algoritmo Aprofundamento Progressivo

In [52]:
mostraProblema(prob4,iterative_deepening_search,'Aprofundamento Progressivo')

= = = = = = = = = = 
= @ N . . . . . . = 
= . = = = = = = . = 
= . = . . . . C . = 
= . = . . . . . . = 
= . = . D . . . . = 
= . = . . . . . . = 
= . . N . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 

Solução  Aprofundamento Progressivo  com custo 10:
Objetivo -->  10
Pontos -->  11
Solução -->  ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W']
Tempo de Execução -->  0.11678894499982562

= = = = = = = = = = 
= @ + + + + + + + = 
= . = = = = = = + = 
= . = . . . . + + = 
= . = . . . . . . = 
= . = . D . . . . = 
= . = . . . . . . = 
= . . N . . . . . = 
= . . . . . . . . = 
= = = = = = = = = = 



##### Observações sobre o problema 3
Neste problema, podemos observar que o único algoritmo que chega a uma solução diferente é Profundidade Primeiro em Grafo e o seu custo é elevado. O tempo de execução varia entre 0.0034s (Largura Primeiro em Grafo) e um ciclo infinito (Profundidade Primeiro em Árvore).

No Algoritmo Profundidade Primeiro em Árvore gera um ciclo infinito uma vez que a procura é iniciada nos ramos mais à esquerda, tendo que expandir e revisitar vários nós, como a solução está nas camadas da superfície mas não nos ramos da esquerda, este pode chegar a uma solução.

Nos algoritmos:
   - Custo Uniforme (0.0048s)
   - Profundidade Primeiro em Grafo (0.0105s) - caminho diferente
   - Aprofundamento Progressivo (0.1167s)
   - Largura Primeiro em Árvore (0.1368s)